
# Árvore binária de busca - Balanceamento (AVL)

- Pacote BinaryTree: https://binarytree.readthedocs.io/en/main/
- https://www.geeksforgeeks.org/binarytree-module-in-python/
- https://pypi.org/project/binarytree/

In [2]:
!pip install binarytree

  10
 /  \
5    15
      \
      20



In [3]:
%%file AVL.c
/*
 * Código fonte árvore do tipo AVL
 */
#include <stdio.h>
#include <stdlib.h>

typedef struct Arvore {
  int info;
  struct Arvore *esq;
  struct Arvore *dir;
} Arv;

typedef struct lista {
  Arv* node;
  struct lista *prox;
} Lista;

typedef struct fila {
  Lista* ini;
  Lista* fim;
} Fila;

Arv *criaNo(int);
Arv *insereAVL(Arv *, int);
int calculaAltura(Arv *);
Arv *abb_retira(Arv *, int);
void print2DTree(Arv *root, int space, int COUNT);
Arv *rotDir(Arv *);
Arv *rotEsq(Arv *);
Arv *rotDirEsq(Arv *);
Arv *rotEsqDir(Arv *);

Arv* removeFila(Fila* pFila){
  if(pFila == NULL)
    return NULL;
  
  Lista* aux = pFila->ini;
  Arv* node;
  if(aux!=NULL){
    pFila->ini = pFila->ini->prox;
    node = aux->node;
    free(aux);    
  }
  //Veirifa se ficou vazio
  if(pFila->ini == NULL)
    pFila->fim = NULL;
  return node;
}

void insereFila(Fila* pFila, Arv* info){
  Lista* novo = (Lista*) malloc(sizeof(Lista));
  novo->node = info;
  novo->prox = NULL;
  if(pFila->ini==NULL){
    pFila->ini = novo;
    pFila->fim = novo;
  }else{
    pFila->fim->prox = novo;
    pFila->fim = novo;
  }  
}

Arv *criaNo(int raiz) {
  Arv *no = (Arv *)malloc(sizeof(Arv));
  no->info = raiz;
  no->esq = NULL;
  no->dir = NULL;
  return no;
}

int calculaAltura(Arv *a) {
  int esq, dir;
  if (a == NULL)
    return -1;

  esq = calculaAltura(a->esq);
  dir = calculaAltura(a->dir);

  if (esq > dir)
    return esq + 1;
  else
    return dir + 1;
}

int getBalanceamento(Arv *no) {
  if (no == NULL)
    return 0;

  return calculaAltura(no->esq) - calculaAltura(no->dir);
}

Arv *rotDir(Arv *no) {
  //printf("\n\nRot Dir >> %d\n", no->info);
  //print2DTree(no, 0, 5);
  Arv *temp;
  Arv *q;
  q = no->esq;
  temp = q->dir;
  q->dir = no;
  no->esq = temp;
  no = q;
  return no;
}

Arv *rotEsq(Arv *no) {
  //printf("\n\nRot Esq >> %d\n", no->info);
  //print2DTree(no, 0, 5);

  Arv *temp;
  Arv *q;
  q = no->dir;
  temp = q->esq;
  q->esq = no;
  no->dir = temp;
  no = q;
  return no;
}

Arv *rotEsqDir(Arv *no) {
  //printf("\n\nRot--Esq--Dir >> %d-->%d\n", no->info, no->esq->info);
  //print2DTree(no, 0, 5);

  no->esq = rotEsq(no->esq);
  return rotDir(no);
}
Arv *rotDirEsq(Arv *no) {
  //printf("\n\nRot--Dir--Esq >> %d-->%d\n", no->info, no->dir->info);
  //print2DTree(no, 0, 5);

  no->dir = rotDir(no->dir);
  return rotEsq(no);
}

Arv *insereAVL(Arv *a, int valor) {
  if (a == NULL)
    return criaNo(valor);

  if (a->info > valor)
    a->esq = insereAVL(a->esq, valor);

  if (a->info < valor)
    a->dir = insereAVL(a->dir, valor);

  int fb = getBalanceamento(a);
  //printf(">>>>> No: %d - Fb: %d\n", a->info, fb);

  
  // Rotação Simples
  ////Direita
  if (fb > 1 && valor < a->esq->info)
    a = rotDir(a);
  ////Esquerda
  if (fb < -1 && valor > a->dir->info)
    a = rotEsq(a);

  // Rotação Dupla
  /// Esquerda-Direita
  if (fb > 1 && valor > a->esq->info)
    a = rotEsqDir(a);
  if (fb < -1 && valor < a->dir->info)
    a = rotDirEsq(a);
  
  /* Retorna a raiz com a nova configuração */
  return a;
}


// Imprime formatado
void print2DTree(Arv *root, int space, int COUNT) {
  // Caso base;
  if (root == NULL)
    return;

  // Aumento da distância entre os níveis
  space += COUNT;

  // Avalia primeiro o nó direita
  // Vai empilhar todas subárvores direitas;
  print2DTree(root->dir, space, COUNT);

  // Imprime o nó no retorno da recursão
  for (int i = COUNT; i < space; i++)
    printf(" ");
  printf("%d\n", root->info);

  // Avalia o nó esquerda
  print2DTree(root->esq, space, COUNT);
}

// Busca em Largura
void BFS(Arv* raiz, FILE* f){
    Arv* node = NULL;
    Fila* F = (Fila*) malloc(sizeof(Fila));
    F->ini = NULL;
    F->fim = NULL;
    insereFila(F, raiz);

    while (F->ini != NULL) {
        node = removeFila(F);
        if(node != NULL){
          //printf("%d, ", node->info);
          fprintf(f, "%d\n",node->info);   
           if (node->esq!=NULL){
              insereFila(F, node->esq);
           }else{
               insereFila(F, NULL);
           }
           if (node->dir!=NULL){
              insereFila(F, node->dir);      
           }else{
              insereFila(F, NULL);
           }  
        }else{
          //printf("%s, ", "None");
          fprintf(f, "%d\n",0); 
        }       
    }      
}

int main(int argc, char **argv){
  FILE *fptr;
  fptr = fopen("bTree.txt", "w");
  
  if (fptr != NULL) {
    printf("Arquivo criado com sucesso!\n");
  }
  else {
    printf("Failed to create the file.\n");
    // exit status for OS that an error occurred
   return -1;
  }  
  
  Arv *root = NULL;
  root = insereAVL(root, 10);
  BFS(root,fptr);
  fprintf(fptr, "%d\n",-1); 
  root = insereAVL(root, 20);
  BFS(root,fptr);
  fprintf(fptr, "%d\n",-1); 
  root = insereAVL(root, 30);
  BFS(root,fptr);
  fprintf(fptr, "%d\n",-1); 
  root = insereAVL(root, 40);
  BFS(root,fptr);
  fprintf(fptr, "%d\n",-1); 
  root = insereAVL(root, 50);
  BFS(root,fptr);
  fprintf(fptr, "%d\n",-1); 
  root = insereAVL(root, 25);
  BFS(root,fptr);
  fclose(fptr);
}


Writing AVL.c


In [5]:
!gcc AVL.c -o avl
!avl

Arquivo criado com sucesso!


In [6]:
!type bTree.txt

10
0
0
-1
10
0
20
0
0
-1
20
10
30
0
0
0
0
-1
20
10
30
0
0
0
40
0
0
-1
20
10
40
0
0
30
50
0
0
0
0
-1
30
20
40
10
25
0
50
0
0
0
0
0
0


In [7]:
## Faz a leitura da arvore gerada em C
# opening the file in read mode

# reading the file
rotations = []
lines = []
with open("bTree.txt", 'r') as f:
     while True:
        line = f.readline().strip()        
        if not line:
            break
        
        if int(line)==-1:          
          rotations.append(lines.copy())          
          lines.clear()
        else:
          lines.append(int(line))

rotations
# printing the data
for i in range(0,len(rotations)):
  for j in range(0, len(rotations[i])): 
    if rotations[i][j] == 0:
      rotations[i][j] = None      
  
print(rotations)
f.close()

[[10, None, None], [10, None, 20, None, None], [20, 10, 30, None, None, None, None], [20, 10, 30, None, None, None, 40, None, None], [20, 10, 40, None, None, 30, 50, None, None, None, None]]


In [8]:
## Gera todas as árvores em .png
# Gerando uma árvore binária
# a partir de uma lista
from binarytree import build2

# Gera o objeto árvore binária
count = 0
graphName = "Rotacao-"
for bTree in rotations:  
  binary_tree = build2(bTree)
  print('Àrvore Binária :\n', binary_tree)
  dfd = binary_tree.graphviz()
  plotName = graphName+str(count)
  dfd.format = 'png'
  dfd.render( filename=plotName)  
  count +=1 

Àrvore Binária :
 
10

Àrvore Binária :
 
10
  \
   20

Àrvore Binária :
 
  _20
 /   \
10    30

Àrvore Binária :
 
  _20
 /   \
10    30
        \
         40

Àrvore Binária :
 
  _20___
 /      \
10      _40
       /   \
      30    50



In [9]:
## Gera arquivo de saida com o teste de balanceamento
### 1: verdadeiro
### 0: falso
from binarytree import build2

#Pega a ABB final gerada
bTree = rotations[len(rotations)-1]
binary_tree = build2(bTree)
print('Àrvore Binária :\n', binary_tree)
   
# Verifica se é Binária de Busca:
if binary_tree.is_bst:
   print("Árvore Binária de Busca!")
else:
   print("Árvore não-ordenada!")

with open('saida.txt', 'w') as f:
   # Verifica se está balanceada:
   if binary_tree.is_balanced:
      print("Árvore Balanceada!")
      f.write('1')
   else:
      print("Árvore não-balanceada!")
      f.write('0')
f.close()  

Àrvore Binária :
 
  _20___
 /      \
10      _40
       /   \
      30    50

Árvore Binária de Busca!
Árvore Balanceada!


In [10]:
%%file teste.c
#include <stdio.h>

int main(int argc, char **argv)
{
  printf("\n%d argumentos\n", argc);
  if(argc>1){
    FILE* ptr = fopen(argv[1], "r");
      if (ptr == NULL) {
          printf("Arquivo não existe!!");
          return 0;
      }
      char temp;
      fscanf(ptr, "%s", &temp);
      printf("Leitura arquivo: %c\n", temp);
  }       
}

Writing teste.c


In [11]:
!gcc teste.c -o teste 
!teste saida.txt


2 argumentos
Leitura arquivo: 1


In [12]:
!type saida.txt

1


### Desenvolver uma função em C que verifica se uma ABB está balanceada. Confirmar a saída da função com o valor o valor recebido do arquivo "saida.txt"

%%file codigo.c

int verificaBalanceamento(Arv*)

-- return 1: se for balanceada

-- return 0: se não for balanceada